Data generation

In [24]:
import numpy as np

# add the path to the abcTau package
import sys
#sys.path.append('./abcTau')
sys.path.append('C:\\Users\\ipochino\\AppData\\Local\\anaconda3\\envs\\isttc\\Lib\\site-packages\\abcTau') # IP: replaced previous line with that; relative path was not working
import abcTau
from scipy.optimize import curve_fit
from statsmodels.tsa.stattools import acf

import os
current_wd = os.getcwd()
os.chdir(os.path.abspath("..\\..\\..\\isttc\\scripts"))
#from calculate_tau import fit_single_exp, func_single_exp_monkey
from cfg_global import project_folder_path
os.chdir(current_wd)

### Generate time series

In [9]:
save_folder = project_folder_path + 'results\\synthetic_data\\dataset\\'

n_signals = 1

In [57]:
tau = np.array([100])
D = 1/tau
deltaT = 1 # 1ms
T = 10*60*1000
numTrials = 1

data_mean = 0.001 # average of firing rate
data_var = 0.002 # variance of firing rate

binSize = 50 # bins size for binning the data for calculating acf
binsData =  np.arange(0, T + binSize, binSize)
numBinData = len(binsData)-1
#print(binsData)
print(numBinData)

# num_lags = 20
# maxTimeLag = int(T/binSize)
# print(maxTimeLag)

12000


In [58]:
for signal_ in range(n_signals):
    ###############
    # OU process 
    ou = abcTau.OU_gen(tau, D, deltaT, T, numTrials)
    print('ou.shape: ', ou.shape)
    
    ou_check = np.max(ou)
    print(ou_check)
    # adjust OU process - scale, shift 
    ou_std = np.sqrt(data_var)
    ou_mean = data_mean
    ou_all = ou_std * ou + ou_mean # this is scale and shift, where is rectify? - no rectify here, this will come for the Poisson rate
    print('ou_all.shape: ', ou_all.shape)
    
    ###############
    # One tau UO with Poisson spike count
    ou_std =  np.sqrt(data_var - data_mean)/deltaT # law of total variance  
    ou_mean = data_mean/deltaT # law of total expectation
    
    # fit mean and var
    ou_poisson = ou_std * ou + ou_mean
    ou_poisson[ou_poisson < 0] = 0 # rectifying
    
    # bin rate and generate spikes
    numBin = int(T/deltaT)
    ou_poisson_rate_ = abcTau.binData(ou_poisson, [numTrials,numBin]) * deltaT
    ou_poisson_rate = np.random.poisson(ou_poisson_rate_)
    
    ###############
    # Going from rate to spikes
    spikeTrain = np.zeros(ou_poisson_rate.shape)
    print(spikeTrain.shape)
    
    for i in range(ou_poisson_rate.shape[0]):
        spikeTrain[i,:] = [1 if ou_poisson_rate[i,j] > np.random.random() else 0 for j in range(ou_poisson_rate.shape[1])]
    
    spike_times = np.where(np.squeeze(spikeTrain) == 1)[0]
    print('N spikes {}'.format(spike_times.shape))
    
    # bin for ACF calculation
    ou_spiketrain_binned = abcTau.binData(spikeTrain, [numTrials, numBinData]) * deltaT
    print(ou_spiketrain_binned.shape)

    ###############
    # Going from rate to spikes - Poisson thinning
    binary_spikeTrain = (np.random.random(ou_poisson_rate.shape) < ou_poisson_rate / np.max(ou_poisson_rate)).astype(int)

    ou_spiketrain_binned_thinned = abcTau.binData(binary_spikeTrain, [numTrials, numBinData]) * deltaT
    print(ou_spiketrain_binned_thinned.shape)

    
    # store
    np.save(save_folder + 'spike_times_' + str(signal_) + '.npy', spike_times)
    np.save(save_folder + 'spike_times_thinned_' + str(signal_) + '.npy', binary_spikeTrain)
    np.save(save_folder + 'spike_times_binned_' + str(signal_) + '.npy', ou_spiketrain_binned)

ou.shape:  (1, 600000)
3.9912667971237683
ou_all.shape:  (1, 600000)
(1, 600000)
N spikes (7677,)
(1, 12000)
(1, 12000)


In [62]:
len(spike_times)/(10*60)

12.795

In [63]:
np.count_nonzero(binary_spikeTrain)/(10*60)

4.266666666666667

In [17]:
binary_spikeTrain.shape

(1, 100000)

#### Check taus

In [64]:
def func_exp_abc_like(x, a, tau):
    return a * np.exp(-x/tau) 

disp = None # put the dispersion parameter if computed with grid-search
#maxTimeLag = 20 # only used when using autocorrelation for summary statistics
summStat_metric = 'comp_cc'
ifNorm = True # if normalize the autocorrelation or PSD
data_sumStat, data_mean, data_var, T, numTrials =  abcTau.preprocessing.extract_stats(ou_poisson_rate, deltaT, binSize,
                                                                                  summStat_metric, ifNorm, maxTimeLag=1000)

# summStat_metric = 'comp_cc'
# ifNorm = True # if normalize the autocorrelation or PSD
# ou_all_data_sumStat, ou_all_data_mean, ou_all_data_var, ou_all_T, ou_all_numTrials =  abcTau.preprocessing.extract_stats(ou_all, deltaT, binSize,
#                                                                                   summStat_metric, ifNorm, maxTimeLag)

print('data_sumStat {}, data_mean {}, data_var {}, T {}, numTrials {}'.format(data_sumStat, data_mean, data_var, T, numTrials))

ou_all_popt, ou_all_pcov = curve_fit(func_exp_abc_like, np.linspace(0,19,20), data_sumStat, maxfev=5000)
print(ou_all_popt)
ou_all_tau_ms = ou_all_popt[1] * binSize
print(ou_all_tau_ms)

data_sumStat [ 1.          0.35231731  0.1983266   0.1001099   0.06176073  0.04460339
  0.02815224  0.0063208  -0.00530664 -0.0097806  -0.02232358 -0.01064589
 -0.01930655 -0.01062005 -0.00846279  0.0128664   0.00367727 -0.00676934
 -0.02459739 -0.03508495], data_mean 0.6518333333333334, data_var 1.3752799722222222, T 600000, numTrials 1
[0.98565689 1.13348638]
56.67431900758323


In [54]:
ou_spiketrain_binned[0].shape

(2000,)

In [65]:
# Using acf func
num_lags = 20
ou_all_acf = acf(ou_spiketrain_binned[0], nlags=num_lags)
print('ou_all_acf: {}'.format(ou_all_acf))

# Using curve_fit
ou_all_popt, ou_all_pcov = curve_fit(func_exp_abc_like, np.linspace(0,20,21), ou_all_acf, maxfev=5000)

ou_all_tau_ms = ou_all_popt[1] * binSize
print(ou_all_tau_ms)

ou_all_acf: [ 1.          0.3527988   0.20088661  0.10281354  0.06249033  0.04495517
  0.02859173  0.00726834 -0.00362977 -0.00789085 -0.02128301 -0.0114812
 -0.01835881 -0.01055834 -0.00898902  0.01338487  0.00256841 -0.00701768
 -0.02570155 -0.03532331 -0.02379843]
57.09269001922627


In [66]:
# Using acf func
num_lags = 20
ou_all_acf = acf(ou_spiketrain_binned_thinned[0], nlags=num_lags)
print('ou_all_acf: {}'.format(ou_all_acf))

# Using curve_fit
ou_all_popt, ou_all_pcov = curve_fit(func_exp_abc_like, np.linspace(0,20,21), ou_all_acf, maxfev=5000)

ou_all_tau_ms = ou_all_popt[1] * binSize
print(ou_all_tau_ms)

ou_all_acf: [ 1.00000000e+00  1.68664228e-01  9.12982538e-02  5.71330272e-02
  5.37415220e-02  2.89169975e-02  3.64930334e-02  1.05009211e-02
  2.71243611e-04 -4.70428894e-03 -2.76848032e-04  5.85558237e-04
 -7.01704678e-03 -2.58960587e-03  2.65125451e-03  8.18401183e-03
 -6.36184842e-03 -2.51820140e-03 -5.68008095e-03 -1.23447238e-02
 -7.10386342e-03]
30.8419397349076


In [35]:
ou_all_tau_ms = ou_all_popt[1] * binSize
print(ou_all_tau_ms)

93.108816189936
